In [1]:
# ==========================================================
# Ensemble Autoencoder + Isolation Forest (con Pretty Logger)
# ==========================================================

import os, random, gc, time
import numpy as np
from sklearn.ensemble import IsolationForest
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import tensorflow as tf

# -------------------------------
# ✅ Setup riproducibilità
# -------------------------------
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

tf.keras.backend.clear_session()
gc.collect()

# -------------------------------
# ✅ Callback grafico training
# -------------------------------
class PrettyTrainingLogger(tf.keras.callbacks.Callback):
    def __init__(self, total_epochs, bar_length=30):
        super().__init__()
        self.total_epochs = total_epochs
        self.bar_length = bar_length
        self.start_time = None

    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        print(f"\n🏗️ Avvio Autoencoder Ensemble...")

    def on_epoch_end(self, epoch, logs=None):
        epoch += 1
        loss = logs.get("loss", 0)
        val = logs.get("val_loss", 0)
        elapsed = time.time() - self.start_time

        progress = epoch / self.total_epochs
        filled = int(self.bar_length * progress)
        bar = "━" * filled + "-" * (self.bar_length - filled)

        print(f"Epoch {epoch}/{self.total_epochs} [{bar}] "
              f"loss={loss:.6f} val={val:.6f} time={elapsed:.2f}s")

# -------------------------------
# 1️⃣ Caricamento dati
# -------------------------------
X_scaled = np.load(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\X_autoencoder_final.npy")
X_latent = np.load(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\X_autoencoder_latent.npy")
print(f"✅ Dati caricati: X_scaled {X_scaled.shape}, X_latent {X_latent.shape}")

# -------------------------------
# 2️⃣ Configurazione ensemble Autoencoder
# -------------------------------
n_ensemble = 3
latent_dim = X_latent.shape[1]
input_dim = X_scaled.shape[1]
ensemble_latent = []
epochs = 10

tf.keras.backend.clear_session()

for i in range(n_ensemble):
    print(f"\n================ Ensemble Model {i+1}/{n_ensemble} ================")

    # Architettura Autoencoder identica
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(32, activation='relu')(encoded)
    encoded = Dense(latent_dim, activation='relu', name='latent_vector')(encoded)
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Logger console training
    logger = PrettyTrainingLogger(total_epochs=epochs)

    autoencoder.fit(
        X_scaled, X_scaled,
        epochs=epochs,
        batch_size=128,
        shuffle=True,
        validation_split=0.1,
        verbose=0,
        callbacks=[logger]
    )

    # Estrazione embeddings
    encoder = Model(inputs=input_layer, outputs=autoencoder.get_layer('latent_vector').output)
    X_latent_i = encoder.predict(X_scaled, verbose=0)
    ensemble_latent.append(X_latent_i)

# -------------------------------
# 3️⃣ Media embeddings ensemble
# -------------------------------
X_ensemble_latent = np.mean(np.stack(ensemble_latent, axis=0), axis=0)
print(f"\n✅ Embeddings ensemble generati: {X_ensemble_latent.shape}")

# -------------------------------
# 4️⃣ Addestramento Isolation Forest
# -------------------------------
iso = IsolationForest(n_estimators=300, contamination='auto', random_state=42)
iso.fit(X_ensemble_latent)

scores = iso.score_samples(X_ensemble_latent)
anomalies = iso.predict(X_ensemble_latent)

# -------------------------------
# 5️⃣ Statistiche
# -------------------------------
n_anomalies = (anomalies == -1).sum()
print(f"📌 Anomalie rilevate: {n_anomalies}/{X_scaled.shape[0]} "
      f"({n_anomalies/X_scaled.shape[0]*100:.2f}%)")

# -------------------------------
# 6️⃣ Salvataggio output
# -------------------------------
np.save(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\X_ensemble_latent.npy", X_ensemble_latent)
np.save(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\anomalies.npy", anomalies)

print("💾 Salvataggio completato.")



✅ Dati caricati: X_scaled (310034, 42), X_latent (310034, 16)

================ Ensemble Model 1/3 ================

🏗️ Avvio Autoencoder Ensemble...
Epoch 1/10 [━━━---------------------------] loss=0.004001 val=0.170712 time=4.37s
Epoch 2/10 [━━━━━━------------------------] loss=0.000177 val=0.175488 time=7.48s
Epoch 3/10 [━━━━━━━━━---------------------] loss=0.000157 val=0.175815 time=10.97s
Epoch 4/10 [━━━━━━━━━━━━------------------] loss=0.000145 val=0.178025 time=14.21s
Epoch 5/10 [━━━━━━━━━━━━━━━---------------] loss=0.000140 val=0.178036 time=17.63s
Epoch 6/10 [━━━━━━━━━━━━━━━━━━------------] loss=0.000135 val=0.178624 time=20.52s
Epoch 7/10 [━━━━━━━━━━━━━━━━━━━━━---------] loss=0.000135 val=0.175581 time=23.46s
Epoch 8/10 [━━━━━━━━━━━━━━━━━━━━━━━━------] loss=0.000132 val=0.179385 time=27.17s
Epoch 9/10 [━━━━━━━━━━━━━━━━━━━━━━━━━━━---] loss=0.000152 val=0.177081 time=30.94s
Epoch 10/10 [━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━] loss=0.000131 val=0.174441 time=35.79s

================ En

In [2]:
# ==========================================================
# Analisi anomalie + associazione label (semi-supervised)
# ==========================================================
import numpy as np
import pandas as pd

# -------------------------------
# 1️⃣ Caricamento embeddings e anomalie
# -------------------------------
X_ensemble_latent = np.load(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\X_ensemble_latent.npy")
anomalies = np.load(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\anomalies.npy")
print(f"✅ Embeddings ensemble: {X_ensemble_latent.shape}, Anomalie: {anomalies.shape}")

# -------------------------------
# 2️⃣ Creazione DataFrame per analisi
# -------------------------------
df_latent = pd.DataFrame(X_ensemble_latent, columns=[f'latent_{i}' for i in range(X_ensemble_latent.shape[1])])

# Associare solo label_technique (array di stringhe)
labels = np.load(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\labels.npy", allow_pickle=True)
df_latent['label_technique'] = labels

# Se non hai label_tactic, puoi mettere 'unknown' oppure creare un array con valori fissi
df_latent['label_tactic'] = ['unknown'] * len(labels)

# Anomalie
df_latent['anomaly'] = anomalies
df_latent['anomaly_flag'] = df_latent['anomaly'].map({1:0, -1:1})  # 1 = anomalia, 0 = normale

print(f"✅ Dataset pronto per analisi esplorativa: {df_latent.shape}")

# -------------------------------
# 3️⃣ Statistiche anomalie per tipo di attacco
# -------------------------------
tech_anomalies = df_latent.groupby('label_technique')['anomaly_flag'].sum()
tech_total = df_latent['label_technique'].value_counts()
tech_pct = (tech_anomalies / tech_total * 100).round(2)

anomaly_summary = pd.DataFrame({
    'total_samples': tech_total,
    'num_anomalies': tech_anomalies,
    '%_anomalous': tech_pct
}).sort_values('%_anomalous', ascending=False)

print("\n📊 Percentuale anomalie per label_technique:")
display(anomaly_summary)

# -------------------------------
# 4️⃣ Salvataggio dataset completo per analisi successive
# -------------------------------
df_latent.to_parquet(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\X_ensemble_latent_labeled.parquet", index=False)
print("💾 Dataset con label e anomalie salvato (parquet).")


✅ Embeddings ensemble: (310034, 16), Anomalie: (310034,)
✅ Dataset pronto per analisi esplorativa: (310034, 20)

📊 Percentuale anomalie per label_technique:


,total_samples,num_anomalies,%_anomalous
label_technique,,,
Other,3119,3119,100.00
T1046,16020,16020,100.00
T1592,20382,20382,100.00
T1595,8104,8104,100.00
T1587,262409,2018,0.77


💾 Dataset con label e anomalie salvato (parquet).


In [3]:
# ==========================================================
# ANALISI ANOMALIE + VALUTAZIONE SEMI-SUPERVISED + DATASET SUPERVISED
# ==========================================================
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# -------------------------------
# 1️⃣ Caricamento embeddings e anomalie
# -------------------------------
X_ensemble_latent = np.load(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\X_ensemble_latent.npy")
anomalies = np.load(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\anomalies.npy")
labels = np.load(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\labels.npy", allow_pickle=True)

print(f"✅ Embeddings ensemble: {X_ensemble_latent.shape}, Anomalie: {anomalies.shape}, Labels: {labels.shape}")

# -------------------------------
# 2️⃣ Creazione DataFrame
# -------------------------------
latent_cols = [f'latent_{i}' for i in range(X_ensemble_latent.shape[1])]
df_latent = pd.DataFrame(X_ensemble_latent, columns=latent_cols)
df_latent['label_technique'] = labels
df_latent['label_tactic'] = ['unknown'] * len(labels)
df_latent['anomaly'] = anomalies
df_latent['anomaly_flag'] = df_latent['anomaly'].map({1:0, -1:1})  # 1 = anomalia

print(f"✅ Dataset pronto: {df_latent.shape}")

# -------------------------------
# 3️⃣ Statistiche anomalie per label_technique
# -------------------------------
tech_anomalies = df_latent.groupby('label_technique')['anomaly_flag'].sum()
tech_total = df_latent['label_technique'].value_counts()
tech_pct = (tech_anomalies / tech_total * 100).round(2)

anomaly_summary = pd.DataFrame({
    'total_samples': tech_total,
    'num_anomalies': tech_anomalies,
    '%_anomalous': tech_pct
}).sort_values('%_anomalous', ascending=False)

print("\n📊 Percentuale anomalie per label_technique:")
display(anomaly_summary)

# -------------------------------
# 4️⃣ Analisi approfondita anomalie
# -------------------------------
df_anomalies = df_latent[df_latent['anomaly_flag'] == 1]
print("\n📌 Numero totale anomalie:", len(df_anomalies))
print("\n📌 Distribuzione anomalie per label_technique:")
display(df_anomalies['label_technique'].value_counts())

# Feature latenti: media anomalie vs normali
mean_features = df_latent.groupby('anomaly_flag')[latent_cols].mean()
print("\n📊 Media feature latenti (anomalia vs normale):")
display(mean_features)

# -------------------------------
# 5️⃣ Valutazione semi-supervised
# -------------------------------
# Attacchi rari = tutte le label diverse da T1587 (dominante)
rare_labels = df_latent['label_technique'].unique().tolist()
if 'T1587' in rare_labels:
    rare_labels.remove('T1587')

y_true = df_latent['label_technique'].apply(lambda x: 1 if x in rare_labels else 0)
y_pred = df_latent['anomaly_flag']

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("\n📌 Metriche semi-supervised (Isolation Forest):")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1-score:  {f1:.3f}")

# -------------------------------
# 6️⃣ Dataset per supervised learning (opzionale)
# -------------------------------
df_supervised = df_latent[df_latent['anomaly_flag'] == 1].copy()
print("\n📌 Dataset solo anomalie per supervised:", df_supervised.shape)

# Dataset bilanciato: anomalie + stesso numero di normali
df_normals = df_latent[df_latent['anomaly_flag'] == 0].sample(n=len(df_supervised), random_state=42)
df_balanced = pd.concat([df_supervised, df_normals]).reset_index(drop=True)
print("📌 Dataset bilanciato per supervised:", df_balanced.shape)

# -------------------------------
# 7️⃣ Salvataggio dataset
# -------------------------------
df_latent.to_parquet(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\X_ensemble_latent_labeled.parquet", index=False)
df_balanced.to_parquet(r"C:\Users\maria\Desktop\Zeek_ML\processed_zeekdata22\X_ensemble_latent_balanced.parquet", index=False)
print("💾 Dataset salvati: completo + bilanciato.")


✅ Embeddings ensemble: (310034, 16), Anomalie: (310034,), Labels: (310034,)
✅ Dataset pronto: (310034, 20)

📊 Percentuale anomalie per label_technique:


,total_samples,num_anomalies,%_anomalous
label_technique,,,
Other,3119,3119,100.00
T1046,16020,16020,100.00
T1592,20382,20382,100.00
T1595,8104,8104,100.00
T1587,262409,2018,0.77



📌 Numero totale anomalie: 49643

📌 Distribuzione anomalie per label_technique:


label_technique
T1592    20382
T1046    16020
T1595     8104
Other     3119
T1587     2018
Name: count, dtype: int64


📊 Media feature latenti (anomalia vs normale):


,latent_0,latent_1,latent_2,latent_3,latent_4,latent_5,latent_6,latent_7,latent_8,latent_9,latent_10,latent_11,latent_12,latent_13,latent_14,latent_15
anomaly_flag,,,,,,,,,,,,,,,,
0,0.594355,0.257848,2.062619,2.827694,1.011494,0.637380,1.703569,1.489913,3.017656,0.584059,3.338214,1.321551,0.485862,2.431566,0.456764,0.923206
1,1.584588,1.867103,1.516809,1.602032,1.413490,0.445802,1.105261,2.729810,2.104894,0.825477,2.629362,2.701282,2.267837,1.791311,2.285766,1.485759



📌 Metriche semi-supervised (Isolation Forest):
Precision: 0.959
Recall:    1.000
F1-score:  0.979

📌 Dataset solo anomalie per supervised: (49643, 20)
📌 Dataset bilanciato per supervised: (99286, 20)
💾 Dataset salvati: completo + bilanciato.
